In [1]:
import sys
sys.path.insert(0, '../')

import pandas as pd
from preprocess import Audio_Processor
from sklearn import metrics
import matplotlib.pyplot as plt
from data_utils import enumerate_strings

Using TensorFlow backend.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
SR = 16000
blocksize = int(SR/2)
overlap = int(SR/4)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Setup

In [4]:
path_to_db='../../ESC-50/'
ps = Audio_Processor(path_to_db + 'audio/', sr=SR)
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50
h_classes = ['Human & Animal', 'Interacting Materials']
mapping = {
    'dog': 0,
    'rooster': 0,
    'pig': 0,
    'cow': 0,
    'frog': 0,
    'cat': 0,
    'hen': 0,
    'insects': 0,
    'sheep': 0,
    'crow': 0,
    'rain': 1,
    'sea_waves': 1,
    'crackling_fire': 1,
    'crickets': 0,
    'chirping_birds': 0,
    'water_drops': 1,
    'wind': 1,
    'pouring_water': 1,
    'toilet_flush': 1,
    'thunderstorm': 1,
    'crying_baby': 0,
    'sneezing': 0,
    'clapping': 1,
    'breathing': 0,
    'coughing': 0,
    'footsteps': 1,
    'laughing': 0,
    'brushing_teeth': 1,
    'snoring': 0,
    'drinking_sipping': 1,
    'door_wood_knock': 1,
    'mouse_click': 1,
    'keyboard_typing': 1,
    'door_wood_creaks': 1,
    'can_opening': 1,
    'washing_machine': 1,
    'vacuum_cleaner': 1,
    'clock_alarm': 1,
    'clock_tick': 1,
    'glass_breaking':1,
    'helicopter': 1,
    'chainsaw': 1,
    'siren': 1,
    'car_horn': 1,
    'engine': 1,
    'train': 1,
    'church_bells': 1,
    'airplane': 1,
    'fireworks': 1,
    'hand_saw': 1,
}
dataset['h_target'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_target'] = mapping[row['category']]

## Getting Preprocessed Data
We allow for previously preprocessed data to be retrieved for faster training turnaround. If the fold has been preprocessed, it is loaded but if not it is processed and saved.

In [5]:
df = ps.preprocess_fold(dataset, 
                        kind='mfcc', 
                        blocksize=blocksize, 
                        overlap=overlap
                       )

In [6]:
df.describe()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sflat_mean,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise,h_target,target
count,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,...,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000,34722.000000
mean,63.941713,65.665808,49.632212,47.134655,39.367883,38.405211,35.596640,34.617313,34.043460,33.051667,...,-0.050182,0.043403,-0.035679,44.485960,42.633316,22.385889,20.838246,14.816204,0.649214,24.570272
std,22.376017,25.318529,16.301524,16.150300,13.220590,12.982169,11.866078,11.593986,11.519877,11.308873,...,0.226584,0.227452,0.223848,3912.281113,1138.943868,823.741668,23.863441,558.972694,0.477223,14.529925
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.799688,-0.818966,-0.829031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.824509,50.057023,40.173008,37.449289,31.674154,30.582568,28.531641,27.543796,27.073887,26.078358,...,-0.217956,-0.119544,-0.201310,7.799437,14.144807,3.960294,14.426031,2.324031,0.000000,12.000000
50%,63.919076,64.288338,50.024940,47.703741,39.806559,39.278398,36.428501,35.475993,34.867461,33.987164,...,-0.054482,0.046015,-0.036857,9.326971,17.195005,4.837236,16.532026,3.292253,1.000000,24.000000
75%,77.246154,80.527143,59.618815,57.470015,47.556487,46.869206,43.462863,42.469852,41.817290,40.769021,...,0.113201,0.212199,0.127219,11.923360,21.039314,5.754140,19.802342,4.699188,1.000000,37.000000
max,194.608514,222.926306,138.137718,121.432810,116.077974,104.890146,85.615582,106.012983,93.750817,83.565897,...,0.763469,0.955219,0.727482,719565.658035,112918.767938,82663.870817,563.421832,67986.785970,1.000000,49.000000


In [7]:
X = df.drop(['target','h_target'], axis=1)
y = df['target']
yy = df['h_target']

# Training

In [8]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline

components = list(range(5,129))

clf = make_pipeline(
    MinMaxScaler(),
    SVC()
)

In [9]:
X.head()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sband_noise,sflat_std,sflat_mean,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise
0,52.009557,4.327045,5.696827,6.522973,3.426770,9.314700,8.897445,1.613144,3.654356,6.125930,...,0.430331,0.430331,-0.430331,-0.430331,0.430331,29.442891,33.986595,7.463287,129.495423,54.772801
1,88.404377,82.645071,55.548627,51.936489,32.090942,23.103625,23.082517,18.536499,16.418501,15.469806,...,-0.164259,0.310696,-0.365104,0.475820,-0.102448,15.695233,20.199985,3.943083,20.152053,1.728330
2,86.987377,78.083572,51.433488,51.561430,33.107803,31.318922,21.869279,15.681414,22.181496,16.827397,...,-0.025999,0.196130,-0.220349,0.254841,0.158864,14.345672,19.755847,3.758124,17.172734,1.683485
3,80.670230,68.750011,19.582458,23.847321,18.158782,20.392203,9.204180,11.268402,12.634953,12.072391,...,0.359768,-0.223875,-0.304151,0.278538,0.105655,25.330133,34.365584,4.991139,27.321271,10.232799
4,81.355627,60.950895,35.026463,48.047609,53.739915,42.014673,48.515288,36.766431,53.463175,52.233060,...,-0.013319,0.086972,0.107285,0.425719,0.243835,12.205491,13.338520,5.419358,15.416913,3.034387


In [10]:
scores = cross_validate(clf, 
                        X, yy, 
                        cv=10, 
                        scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
                        n_jobs=-1
                       )

In [11]:
default_stats = pd.DataFrame(scores)
default_stats.head()

,fit_time,score_time,test_accuracy,test_f1,test_precision,test_recall,test_roc_auc,train_accuracy,train_f1,train_precision,train_recall,train_roc_auc
0,247.269292,67.344104,0.649295,0.787360,0.649295,1.000000,0.727331,0.649109,0.787224,0.649171,0.999852,0.722829
1,248.705145,67.578444,0.647855,0.786226,0.648875,0.997339,0.666216,0.649045,0.787069,0.649273,0.999113,0.730052
2,210.093337,67.036480,0.649194,0.787286,0.649194,1.000000,0.728762,0.649216,0.787303,0.649216,1.000000,0.724480
3,247.223046,67.512124,0.647465,0.786014,0.648586,0.997338,0.735811,0.649216,0.787303,0.649216,1.000000,0.723236
4,247.682037,68.162613,0.649194,0.787286,0.649194,1.000000,0.676631,0.648992,0.787113,0.649166,0.999507,0.725205


In [12]:
default_stats.mean()

fit_time           243.401737
score_time          67.972041
test_accuracy        0.649127
test_f1              0.787170
test_precision       0.649261
test_recall          0.999468
test_roc_auc         0.703478
train_accuracy       0.649143
train_f1             0.787209
train_precision      0.649235
train_recall         0.999655
train_roc_auc        0.725399
dtype: float64

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('scaler', MinMaxScaler()),
    ('feat', PCA(n_components=None)),
    ('classify', SVC())
])

params = {
    'feat__n_components': range(25, 100, 25)
}

gridsrc = GridSearchCV(clf, 
                        cv=10, 
                        param_grid=params,
                        scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
                        verbose=2,
                        n_jobs=5,
                        refit=False
                       )
gridsrc.fit(X,yy)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  30 out of  30 | elapsed: 36.5min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('feat', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classify', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'feat__n_components': range(25, 100, 25)},
       pre_dispatch='2*n_jobs', refit=False, return_train_score='warn',
       scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
       verbose=2)

In [15]:
pca_stats = pd.DataFrame(gridsrc.cv_results_)
pca_stats.head()

,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_f1,mean_test_precision,mean_test_recall,mean_test_roc_auc,mean_train_accuracy,mean_train_f1,mean_train_precision,...,std_test_accuracy,std_test_f1,std_test_precision,std_test_recall,std_test_roc_auc,std_train_accuracy,std_train_f1,std_train_precision,std_train_recall,std_train_roc_auc
0,77.836667,16.855571,0.648638,0.786628,0.649296,0.997649,0.685621,0.649758,0.787266,0.649913,...,0.001051,0.001156,0.000377,0.004037,0.040387,0.000527,0.000395,0.001005,0.003497,0.003901
1,80.808460,25.822940,0.654225,0.786461,0.656540,0.980880,0.703134,0.662862,0.791599,0.661385,...,0.004926,0.002282,0.006265,0.016555,0.041584,0.009178,0.002611,0.008953,0.012419,0.002135
2,118.483286,37.087304,0.652094,0.786890,0.653270,0.989398,0.703929,0.657841,0.790252,0.656441,...,0.003309,0.001483,0.004063,0.010918,0.037153,0.006274,0.002006,0.005724,0.007584,0.002504


In [16]:
for col in pca_stats.columns:
    print(col)

mean_fit_time
mean_score_time
mean_test_accuracy
mean_test_f1
mean_test_precision
mean_test_recall
mean_test_roc_auc
mean_train_accuracy
mean_train_f1
mean_train_precision
mean_train_recall
mean_train_roc_auc
param_feat__n_components
params
rank_test_accuracy
rank_test_f1
rank_test_precision
rank_test_recall
rank_test_roc_auc
split0_test_accuracy
split0_test_f1
split0_test_precision
split0_test_recall
split0_test_roc_auc
split0_train_accuracy
split0_train_f1
split0_train_precision
split0_train_recall
split0_train_roc_auc
split1_test_accuracy
split1_test_f1
split1_test_precision
split1_test_recall
split1_test_roc_auc
split1_train_accuracy
split1_train_f1
split1_train_precision
split1_train_recall
split1_train_roc_auc
split2_test_accuracy
split2_test_f1
split2_test_precision
split2_test_recall
split2_test_roc_auc
split2_train_accuracy
split2_train_f1
split2_train_precision
split2_train_recall
split2_train_roc_auc
split3_test_accuracy
split3_test_f1
split3_test_precision
split3_test_recal

In [17]:
pca_stats_red = pca_stats[
    ['param_feat__n_components', 
     'mean_test_roc_auc', 
     'mean_test_accuracy', 
     'mean_test_recall', 
     'mean_test_precision']
]

pca_stats_red.sort_values(
             ['mean_test_roc_auc', 
              'mean_test_accuracy', 
              'mean_test_recall', 
              'mean_test_precision']
             , ascending=False)

,param_feat__n_components,mean_test_roc_auc,mean_test_accuracy,mean_test_recall,mean_test_precision
2,75,0.703929,0.652094,0.989398,0.653270
1,50,0.703134,0.654225,0.980880,0.656540
0,25,0.685621,0.648638,0.997649,0.649296


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('scaler', MinMaxScaler()),
    ('feat', KernelPCA(n_components=None)),
    ('classify', SVC())
])

params = {
    'feat__n_components': range(25, 100, 25),
    'feat__kernel': ['poly', 'rbf', 'sigmoid', 'cosine']
}

gridsrc = GridSearchCV(clf, 
                        cv=10, 
                        param_grid=params,
                        scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
                        verbose=2,
                        n_jobs=5,
                        refit=False
                       )
gridsrc.fit(X,yy)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


Fitting 10 folds for each of 12 candidates, totalling 120 fits


In [ ]:
kpca_stats = pd.concat(dfs, ignore_index=True)
kpca_stats.head()

In [ ]:
kpca_stats.groupby(['kernel','components']).mean().sort_values(['test_accuracy', 'test_recall', 'test_precision'], ascending=False)